In [21]:
import json
import pandas as pd
from collections import Counter
import ast
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [4]:
f = open('dataset/wiki_scraping/wiki_texts.json')
wiki_texts = json.load(f)

In [5]:
big_df = pd.read_csv('big_df.csv')

In [6]:
big_df.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
big_df['tknzd_lyrics'] = big_df['tknzd_lyrics'].apply(ast.literal_eval)
big_df['emotions'] = big_df['emotions'].apply(ast.literal_eval)
big_df['social_tags'] = big_df['social_tags'].apply(ast.literal_eval)

In [8]:
drop_rows = []
for row in big_df.iterrows():
    if row[1]['emotions'] == []:
        drop_rows.append(row[0])
big_df.drop(drop_rows, inplace=True)

In [69]:
big_df

,title,tag,artist,year,views,features,lyrics,id,song_id,emotions,social_tags,tknzd_lyrics
2,A Milli,rap,Lil Wayne,2008,1237174,{},[Intro]\nBangladesh\nYoung Money!\nYou dig?\nM...,59,TRFASZI128F93229B7,"[(party, 1)]","[(hip-hop, 100), (rap, 87), (lil wayne, 78), (...","[bangladesh, young, money, you, dig, mack, im,..."
3,The Documentary,rap,The Game,2005,236542,{},"[Intro: Livin' tha Life (2003)]\nBitch, I want...",63,TRECLHK128E0784A19,"[(tragic, 7)]","[(rap, 100), (hip-hop, 71), (the game, 71), (t...","[bitch, i, want, my, mother, fuckin, money, ba..."
4,My Name Is,rap,Eminem,1999,1464039,{},"[Produced by Dr. Dre]\n\n[Chorus: Eminem]\nHi,...",68,TROVBAL12903CC4F1B,"[(funny, 4), (fun, 2), (party, 1)]","[(rap, 100), (hip-hop, 73), (eminem, 65), (hip...","[hi, my, name, is, what, my, name, is, who, my..."
5,We Fly High,rap,Jim Jones,2006,63337,{},[Intro: Sample & Jim Jones]\nI wear a mean dar...,80,TRTJQTZ128F92F4A64,"[(party, 6), (happy, 3)]","[(hip-hop, 100), (rap, 93), (hip hop, 50), (ba...","[i, wear, a, mean, dark, pair, of, shades, and..."
9,Keys Open Doors,rap,Clipse,2006,61909,{},"[Chorus: Pusha T]\nKeys open doors, keys open ...",116,TRBSYSE128F42BBDCF,"[(dark, 16), (smooth, 8), (tense, 8)]","[(hip-hop, 100), (the neptunes, 58), (neptunes...","[keys, open, doors, keys, open, doors, keys, o..."
...,...,...,...,...,...,...,...,...,...,...,...,...
122569,Taste For Harmony,pop,Union Of Knives,2006,5,{},Trust me\nI know where I'm going\nIt won't be\...,7817131,TRCAWPX128F1479233,"[(melancholic, 50)]","[(trylove, 100), (love, 100), (indie, 100), (t...","[trust, me, i, know, where, im, going, it, won..."
122585,Gravity,pop,Brenda Russell,1988,2,{},[Verse 1: Brenda]\nIf it's love that brought u...,7840324,TRXCBHR128F42B226B,"[(fun, 33)]","[(soul, 100), (groovy, 33), (fun, 33), (love s...","[if, its, love, that, brought, us, together, w..."
122592,Floating Free,pop,Vibrasphere,2010,1,{},I wanna do it\nI'm gonna do it\nI wanna do it\...,7852356,TRDERZZ128F93350F5,"[(sexy, 3), (relax, 3)]","[(progressive trance, 100), (psytrance, 83), (...","[i, wanna, do, it, im, gonna, do, it, i, wanna..."
122606,Grey Matter,pop,Jewel,2001,8,{},"[Intro]\nI hate you, I love you\nLeave, don't ...",7870813,TRLMRUW128F9330A10,"[(calm, 25)]","[(alternative, 100), (pop, 50), (love, 25), (f...","[i, hate, you, i, love, you, leave, dont, go, ..."


In [9]:
all_words = Counter()
c = 0
for lyr in big_df['tknzd_lyrics']:
    c+=1
    if c %1000 == 0:
        print(c)
    all_words = all_words +  Counter(lyr)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000


In [10]:
all_words2 = {}
for w in all_words:
    c = all_words[w]
    if c>10 and c <47000:
        all_words2[w] = c

In [11]:
all_words = {}

In [12]:
word_to_index = {w:i for i, w in enumerate(all_words2)}

In [13]:
index_to_word = {i:w for i, w, in enumerate(all_words2)}

In [45]:
song_tf_idf = np.zeros((big_df.shape[0], len(all_words2))) #i, j how many times word j appears in song i
for i, lyr in enumerate(big_df['tknzd_lyrics']):
    for wrd in lyr:
        if wrd in word_to_index:
            j = word_to_index[wrd]
            song_tf_idf[i, j] += 1

In [46]:
song_tf_idf = (song_tf_idf + 0.5) / (song_tf_idf.sum(axis=0) + 0.5)

In [47]:
song_tf_idf

array([[0.00104058, 0.00131703, 0.00182593, ..., 0.04347826, 0.04      ,
        0.02702703],
       [0.00020812, 0.00094073, 0.00060864, ..., 0.04347826, 0.04      ,
        0.02702703],
       [0.00020812, 0.00018815, 0.00060864, ..., 0.04347826, 0.04      ,
        0.02702703],
       ...,
       [0.00020812, 0.00018815, 0.00060864, ..., 0.04347826, 0.04      ,
        0.02702703],
       [0.00020812, 0.00018815, 0.00060864, ..., 0.04347826, 0.04      ,
        0.02702703],
       [0.00020812, 0.00018815, 0.00060864, ..., 0.04347826, 0.04      ,
        0.02702703]])

In [48]:
with open('word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f)
with open('index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f)

In [49]:
with open('song_tf_idf.npy', 'wb') as f:
    np.save(f, song_tf_idf)

In [70]:
song_to_index = {s:i for i, s in enumerate(big_df['title'])}
index_to_song = {i:s for i, s in enumerate(big_df['title'])}

In [71]:
with open('song_to_index.pkl', 'wb') as f:
    pickle.dump(song_to_index, f)
with open('index_to_song.pkl', 'wb') as f:
    pickle.dump(index_to_song, f)

# wiki

In [64]:
from dataset import wiki_scraping

In [65]:
wiki_tf_idf = np.zeros((len(wiki_texts), len(all_words2)))
for i, city in enumerate(wiki_texts.keys()):
    txt = wiki_texts[city]
    for wrd in txt:
        if wrd in word_to_index:
            j = word_to_index[wrd]
            wiki_tf_idf[i, j] += 1

In [66]:
wiki_tf_idf = (wiki_tf_idf + 0.5) / (wiki_tf_idf.sum(axis=0) + 0.5)

In [67]:
loc_to_index = {cty:i for i, cty in enumerate(wiki_texts.keys())}

In [68]:
wiki_tf_idf.shape

(44, 18502)